In [1]:
'''
ROAD LANE LINE DETECTION SYSTEM

Dataset used: TUSimple
Datasetlink: https://www.kaggle.com/datasets/manideep1108/tusimple
(23GB Dataset size)
Assign paths according to location on device
'''

'\nROAD LANE LINE DETECTION SYSTEM\n\nDataset used: TUSimple\nDatasetlink: https://www.kaggle.com/datasets/manideep1108/tusimple\n(23GB Dataset size)\nAssign paths according to location on device\n'

In [ ]:
# Importing neccesary libraries
import os
import cv2
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as pl
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [3]:
# Assigning paths
data='/home/pranjaysaxena/Downloads/archive/TUSimple/train_set'
label='/home/pranjaysaxena/Downloads/archive/TUSimple/train_set/label_data_0601.json'

In [4]:
with open(label,'r') as f:
  labels = [json.loads(line) for line in f]

In [5]:
#Preprocessing data(images & labels)
def preprocess_data(data,labels,img_size=(256,256)):
  images = []
  layers =[]
  for label in labels:
    img_path = os.path.join(data,label['raw_file'])
    img = cv2.imread(img_path)
    img = cv2.resize(img, img_size)
    images.append(img)
    layer = np.zeros((720, 1280), dtype=np.uint8)
    lanes = label['lanes']
    h_samples = label['h_samples']
    for lane in lanes:
      if len(lanes) > 0:
        points = [(x,y) for x,y in zip(lane, h_samples) if x >= 0]
        for i in range(len(points) - 1):
          cv2.line(layer, points[i],points[i+1], 255, 5)
    layer = cv2.resize(layer, img_size)
    layers.append(layers)

  return np.array(images), np.array(layers)

In [ ]:
# Preprocessing the data
images, layers = preprocess_data(data, labels)
#Split data into training and evaluation sets
X_train, X_eval, y_train, y_eval = train_test_split(images, layers, test_size=0.2, random_state=42)

In [ ]:
# Normalization Step
X_train = X_train / 255.0
X_eval = X_eval / 255.0
y_train = y_train / 255.0
y_eval = y_eval / 255.0

# Dimension expansion of layers to match model outut shape
y_train = np.expand_dims(y_train, axis=-1)
y_eval = np.expand_dims(y_eval, axis=-1)

In [ ]:
# Model Defination
def build_model(input_shape):
  inputs = layers.Input(shape=input_shape)

  conv1 = layers.Conv2D(32,(3,3),activation='relu',padding='same')(inputs)
  pool1 = layers.MaxPooling2D((2,2))(conv1)

  conv2 = layers.Conv2D(64,(3,3),activation='relu',padding='same')(pool1)
  pool2 = layers.MaxPooling2D((2,2))(conv2)

  conv3 = layers.Conv2D(128,(3,3),activation='relu',padding='same')(pool2)
  pool3 = layers.MaxPooling2D((2,2))(conv3)

  conv4 = layers.Conv2D(256,(3,3),activation='relu',padding='same')(pool3)

  up1 = layers.Conv2DTranspose(128,(3,3),strides=(2,2),padding='same')(conv4)
  concat1 = layers.concatenate([conv3,up1],axis=-1)
  conv5 = layers.Conv2D(128,(3,3),activation='relu',padding='same')(concat1)

  up2 = layers.Conv2DTranspose(64,(3,3),strides=(2,2),padding='same')(conv5)
  concat2 = layers.concatenate([conv2,up2],axis=-1)
  conv6 = layers.Conv2D(64,(3,3),activation='relu',padding='same')(concat2)

  up3 = layers.Conv2DTranspose(32,(3,3),strides=(2,2),padding='same')(conv6)
  concat3 = layers.concatenate([conv1,up3],axis=-1)
  conv7 = layers.Conv2D(32,(3,3),activation='relu',padding='same')(concat3)

  outputs = layers.Conv2D(1,(1,1),activation='sigmoid')(conv7)

  model = models.Model(inputs,outputs)
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [ ]:
# Model Building
input_shape = (256,256,3)
model = build_model(input_shape)
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_eval, y_eval), verbose=1)

# Plot training & evaluation accuracies
plt.plot(history.history['accuracy'])
plt.plot(history.history['eval_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Evaluation'], loc='upper left')
plt.show()

# Plot training & evaluation loss data
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Evaluation'], loc='upper left')
plt.show()

# Testing Model
eval_loss,eval_acc = model.evaluate(X_eval,y_eval,verbose=2)
print(f'Evaluation Loss: {eval_loss}')
print(f'Evaluation Accuracy: {eval_acc}')

# Predict on a few validation images
predictions = model.predict(X_eval)

# Plot results
def plot_sample(i):
  plt.figure(figsize=(10,5))
  plt.subplot(1,3,1)
  plt.title('Input Image')
  plt.imshow(X_eval[i])

  plt.subplot(1,3,2)
  plt.title('Ground Truth')
  plt.imshow(y_eval[i].reshape(256,256),cmap='gray')

  plt.subplot(1,3,3)
  plt.title('Prediction')
  plt.imshow(predictions[i].reshape(256,256),cmap='gray')

  plt.show()


In [ ]:
# Ploting a few samples
for i in range(5):
  plot_sample(i)